In [1]:
import pandas as pd
import re
from gensim import corpora,models,similarities
from gensim.utils import tokenize
from gensim.parsing.preprocessing import remove_stopwords

def tokenize_in_df(strin):
    try:
        return list(
            tokenize(
                strin,
            )
        )
    except:
        return ""


In [2]:
df = pd.read_csv("base.csv", sep=",", names=["names", "text"])[["names", "text"]]
df_1 = pd.read_csv("to_search_for.csv", sep=",", names=["names", "source", "text"])[
    ["names", "source", "text"]
]
df["tokens"] = df["text"].apply(tokenize_in_df)

In [24]:
dictionary = corpora.Dictionary(df["tokens"])
feature_cnt = len(dictionary.token2id)
corpus = [dictionary.doc2bow(text) for text in df["tokens"]]
tfidf = models.TfidfModel(corpus, smartirs='dtu')
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=feature_cnt)

In [25]:
for text in df_1["text"]:
    kw_vector = dictionary.doc2bow(tokenize(text))
    df[text] = index[tfidf[kw_vector]]

In [28]:
df

names  \
6      ./repos/tmate/cmd-queue.c   
64          ./repos/tmate/utf8.c   
45     ./repos/tmate/arguments.c   
8      ./repos/tmate/cmd-queue.c   
7      ./repos/tmate/cmd-queue.c   
..                           ...   
22          ./repos/tmate/proc.c   
36        ./repos/tmate/alerts.c   
21          ./repos/tmate/proc.c   
27  ./repos/tmate/window-clock.c   
29  ./repos/tmate/window-clock.c   

                                                 text  \
6   void\ncmdq_print(struct cmd_q *cmdq, const cha...   
64  char *\nutf8_sanitize(const char *src)\n{\n\tc...   
45  int\nargs_cmp(struct args_entry *a1, struct ar...   
8   void\ncmdq_guard(struct cmd_q *cmdq, const cha...   
7   void\ncmdq_error(struct cmd_q *cmdq, const cha...   
..                                                ...   
22  void\nproc_exit(struct tmuxproc *tp)\n{\n\ttp-...   
36  void\nalerts_check_session(struct session *s)\...   
21  void\nproc_loop(struct tmuxproc *tp, int (*loo...   
27  struct screen *\nwindow_clock_init(struct wind...   
29  void\nwindow_clock_resize(struct window_pane *...   

                                               tokens  \
6   [void, cmdq_print, struct, cmd_q, cmdq, const,...   
64  [char, utf, _sanitize, const, char, src, char,...   
45  [int, args_cmp, struct, args_entry, a, struct,...   
8   [void, cmdq_guard, struct, cmd_q, cmdq, const,...   
7   [void, cmdq_error, struct, cmd_q, cmdq, const,...   
..                                                ...   
22  [void, proc_exit, struct, tmuxproc, tp, tp, exit]   
36  [void, alerts_check_session, struct, session, ...   
21  [void, proc_loop, struct, tmuxproc, tp, int, l...   
27  [struct, screen, window_clock_init, struct, wi...   
29  [void, window_clock_resize, struct, window_pan...   

    int strvis(undefined2 *param_1,char *param_2,uint param_3)\n\n{\n  char *pcVar1;\n  char cVar2;\n  undefined2 *puVar3;\n  \n  cVar2 = *param_2;\n  puVar3 = param_1;\n  if (cVar2 == '\0') {\n    *(undefined *)param_1 = 0;\n    return 0;\n  }\n  do {\n    pcVar1 = param_2 + 1;\n    param_2 = param_2 + 1;\n    puVar3 = vis(puVar3,(int)cVar2,param_3,*pcVar1);\n    cVar2 = *param_2;\n  } while (cVar2 != '\0');\n  *(undefined *)puVar3 = 0;\n  return (int)puVar3 - (int)param_1;\n}  \
6                                            0.024490                                                                                                                                                                                                                                                                                                                                                                                                                                                
64                                           0.092114                                                                                                                                                                                                                                                                                                                                                                                                                                                
45                                           0.025113                                                                                                                                                                                                                                                                                                                                                                                                                                                
8                                            0.041101                                                                                                                                                                                                                                   

In [27]:
for i in range(df_1.shape[0]):
    df = df.sort_values(by=df_1["text"][i], ascending=False)
    print(
        "\nThe "
        + df_1["names"][i]
        + " function is probably contained in one of these files:\n"
    )
    print(df.head(3)["names"].to_string(index=False))

    print("\nActual source:\n" + df_1["source"][i])



The strvis function is probably contained in one of these files:

     ./repos/tmate/utf8.c
     ./repos/tmate/proc.c
./repos/tmate/cmd-queue.c

Actual source:
./repos/tmate/utf8.c

The alerts_ring_bell function is probably contained in one of these files:

   ./repos/tmate/alerts.c
./repos/tmate/arguments.c
./repos/tmate/cmd-queue.c

Actual source:
./repos/tmate/alerts.c

The cmdq_print function is probably contained in one of these files:

./repos/tmate/cmd-queue.c
     ./repos/tmate/utf8.c
./repos/tmate/arguments.c

Actual source:
./repos/tmate/cmd-queue.c
